In [ ]:
import pandas as pd

def combine_model_responses(file_model_pairs):
    """
    Combines responses from different files based on specified model columns.
    
    Args:
        file_model_pairs (dict): Dictionary mapping filenames to their response column names
    """
    combined_data = None
    
    for file, column in file_model_pairs.items():
        df = pd.read_csv(file)
        
        # Select Prompt and specific model response
        selected_data = df[['Prompt', column]].copy()
        selected_data = selected_data.rename(columns={column: f"{file.split('_')[1].split('.')[0]}_response"})
        
        if combined_data is None:
            combined_data = selected_data
        else:
            combined_data = pd.merge(combined_data, selected_data, on='Prompt', how='outer')
    
    return combined_data

# Example usage
file_model_pairs = {
        '../results/results_chatgpt.csv': 'GPTResponse',
        '../results/results_claude.csv': 'ClaudeResponse',
        '../results/results_llama31.csv': 'LLama31Response',
        '../results/results_llama32.csv': 'LLama32Response',
        '../results/results_qwen.csv': 'QwenResponse',
        '../results/results_phi.csv': 'PhiResponse',
        '../results/results_smoll.csv': 'SmollResponse',



}

combined_responses = combine_model_responses(file_model_pairs)
combined_responses.to_csv('../results/combined_model_responses.ssv', index=False, sep=";")

KeyError: "['LLama31Response'] not in index"